# Optimal control of double pendulum

In [1]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

## The Problem

We have a double pendulum with masses $m_1=1$ and $m_2=m$ and with lengths $L_1$ and $L_2$ which is released from an arbitrary point and has to be driven to the target position.

The cartesian coordinates of the center of mass of each point are defined as
$$
\begin{align}
x_1 =& L_1 \sin(\theta_1) \\
y_1 =& -L_1 \cos(\theta_1) \\
x_2 =& x_1 + L_2 \sin(\theta_2) = L_1 \sin(\theta_1) + L_2 \sin(\theta_2)\\
y_2 =& y_1 - L_2 \cos(\theta_2)= -L_1 \cos(\theta_1)-L_2 \cos(\theta_2)
\end{align}
$$

In [ ]:
# define coordination function
x1_fn = lambda l_1, the_1: l_1 * jnp.sin(the_1)
y1_fn = lambda l_1, the_1: - l_1 * jnp.cos(the_1)

x2_fn = lambda l_1, the_1, l_2, the_2: l_1 * jnp.sin(the_1) + l_2 * jnp.sin(the_2)
y2_fn = lambda l_1, the_1, l_2, the_2: - l_1 * jnp.cos(the_1) - l_2 * jnp.cos(the_2)

The dynamics of the double pendulum in polar coordinates are given by the following equations:
    
$$
\begin{align}
\dot{\theta}_1 =& z_1 \\
\dot{z}_1 =& \ddot{\theta}_1 = \dots \\
\dot{\theta}_2 =& z_2 \\
\dot{z}_2 =& \ddot{\theta}_2 = \dots
\end{align}
$$

$$
\begin{align*}
\dot{z_{1}} =& \frac{L_1 m \sin(\theta_1(t) - \theta_2(t)) \cos(\theta_1(t) - \theta_2(t)) \left( \frac{d\theta_1(t)}{dt} \right)^2}
{L_1 m \cos^2(\theta_1(t) - \theta_2(t)) - L_1 m - L_1} \\
& \quad + \frac{L_2 m \sin(\theta_1(t) - \theta_2(t)) \left( \frac{d\theta_2(t)}{dt} \right)^2}
{L_1 m \cos^2(\theta_1(t) - \theta_2(t)) - L_1 m - L_1} \\
& \quad + \frac{g m \sin(\theta_1(t))}{L_1 m \cos^2(\theta_1(t) - \theta_2(t)) - L_1 m - L_1} \\
& \quad - \frac{g m \sin(\theta_2(t)) \cos(\theta_1(t) - \theta_2(t))}{L_1 m \cos^2(\theta_1(t) - \theta_2(t)) - L_1 m - L_1} \\
& \quad + \frac{g \sin(\theta_1(t))}{L_1 m \cos^2(\theta_1(t) - \theta_2(t)) - L_1 m - L_1}.
\end{align*}
$$

$$
\begin{align*}
\dot{z_{2}} =
&- \frac{L_1 m \sin(\theta_1(t) - \theta_2(t)) \left(\frac{d \theta_1(t)}{dt}\right)^2}{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} \\
&- \frac{L_1 \sin(\theta_1(t) - \theta_2(t)) \left(\frac{d \theta_1(t)}{dt}\right)^2 }{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} \\
&- \frac{L_2 m \sin(\theta_1(t) - \theta_2(t)) \cos(\theta_1(t) - \theta_2(t)) \left(\frac{d \theta_2(t)}{dt}\right)^2}{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} \\
&- \frac{g m \sin(\theta_1(t)) \cos(\theta_1(t) - \theta_2(t))}{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} \\
&+ \frac{g m \sin(\theta_2(t))}{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} \\
&- \frac{g \sin(\theta_1(t)) \cos(\theta_1(t) - \theta_2(t))}{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} \\
&- \frac{g \sin(\theta_2(t))}{L_2 m \cos^2(\theta_1(t) - \theta_2(t)) - L_2 m - L_2} 
\end{align*}
$$

In [ ]:
def z1_fn(state, u):
    l_1, the_1, l_2, the_2 = state
    l_1_dot, the_1_dot, l_2_dot, the_2_dot = u
    return jnp.array([
        l_1_dot,
        the_1_dot,
        l_2_dot,
        the_2_dot
    ])